<a href="https://colab.research.google.com/github/neetushibu/IontheFold-Team6/blob/main/Enhanced_Charge_Analyzer_Resume_FAST_SAFE_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import gzip
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import re
from typing import List, Tuple, Dict
from scipy.spatial.distance import cdist
from collections import defaultdict
from datetime import datetime
import glob
import json
import csv
import threading
import time
import pandas as pd

In [3]:
class EnhancedChargeAffinityAnalyzer:
    def __init__(self):
        self.atoms = []
        self.bonds = []
        self.filename = ""
        self.charged_residues = []
        self.binding_sites = []
        self.sequence_data = {}

        # Real-time file writing setup
        self.output_writers = {}
        self.output_files = {}
        self.write_lock = threading.Lock()

    def open_file(self, filepath: str) -> str:
        """Open and read a file, handling both compressed and uncompressed formats."""
        self.filename = os.path.basename(filepath)

        try:
            if filepath.endswith('.gz'):
                with gzip.open(filepath, 'rt', encoding='utf-8') as f:
                    content = f.read()
                print(f"✓ Successfully opened compressed file: {self.filename}")
            else:
                with open(filepath, 'r', encoding='utf-8') as f:
                    content = f.read()
                print(f"✓ Successfully opened file: {self.filename}")

            return content

        except Exception as e:
            print(f"✗ Error opening file {filepath}: {e}")
            return ""

    def extract_sequence_from_pdb(self, content: str) -> Dict:
        """Extract protein sequences from PDB content."""
        lines = content.split('\n')
        sequences = {}
        current_chain = None

        # Standard amino acid mapping
        aa_mapping = {
            'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C',
            'GLU': 'E', 'GLN': 'Q', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I',
            'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P',
            'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'
        }

        # First try to get sequence from SEQRES records (preferred)
        for line in lines:
            if line.startswith('SEQRES'):
                chain_id = line[11].strip()
                if chain_id not in sequences:
                    sequences[chain_id] = {'seqres': '', 'atom_based': '', 'length': 0}

                # Extract residues from SEQRES line
                residues = line[19:].strip().split()
                for res in residues:
                    if res in aa_mapping:
                        sequences[chain_id]['seqres'] += aa_mapping[res]
                    else:
                        sequences[chain_id]['seqres'] += 'X'  # Unknown residue

        # Also extract sequence from ATOM records as backup
        chain_residues = {}
        for line in lines:
            if line.startswith('ATOM'):
                try:
                    chain = line[21].strip()
                    residue_name = line[17:20].strip()
                    residue_id = int(line[22:26].strip()) if line[22:26].strip() else 0

                    if chain not in chain_residues:
                        chain_residues[chain] = {}

                    if residue_id not in chain_residues[chain]:
                        chain_residues[chain][residue_id] = residue_name

                except (ValueError, IndexError):
                    continue

        # Build atom-based sequences
        for chain_id, residues in chain_residues.items():
            if chain_id not in sequences:
                sequences[chain_id] = {'seqres': '', 'atom_based': '', 'length': 0}

            sorted_residues = sorted(residues.items())
            atom_seq = ''
            for res_id, res_name in sorted_residues:
                if res_name in aa_mapping:
                    atom_seq += aa_mapping[res_name]
                else:
                    atom_seq += 'X'
            sequences[chain_id]['atom_based'] = atom_seq

        # Finalize sequences (prefer SEQRES, fallback to atom-based)
        for chain_id in sequences:
            final_seq = sequences[chain_id]['seqres'] or sequences[chain_id]['atom_based']
            sequences[chain_id]['final_sequence'] = final_seq
            sequences[chain_id]['length'] = len(final_seq)

        self.sequence_data = sequences
        print(f"✓ Extracted sequences for {len(sequences)} chains")
        for chain_id, data in sequences.items():
            print(f"  Chain {chain_id}: {data['length']} residues")

        return sequences

    def parse_pdb(self, content: str) -> None:
        """Parse PDB format file content and extract sequences."""
        self.atoms = []

        # Extract sequences first
        self.extract_sequence_from_pdb(content)

        # Parse atoms
        lines = content.split('\n')
        for line in lines:
            if line.startswith('ATOM') or line.startswith('HETATM'):
                try:
                    atom_id = int(line[6:11].strip())
                    atom_name = line[12:16].strip()
                    residue_name = line[17:20].strip()
                    chain = line[21].strip()
                    residue_id = int(line[22:26].strip()) if line[22:26].strip() else 0
                    x = float(line[30:38].strip())
                    y = float(line[38:46].strip())
                    z = float(line[46:54].strip())
                    element = line[76:78].strip() or atom_name[0]

                    self.atoms.append({
                        'id': atom_id,
                        'name': atom_name,
                        'element': element,
                        'residue': residue_name,
                        'chain': chain,
                        'residue_id': residue_id,
                        'x': x, 'y': y, 'z': z
                    })
                except (ValueError, IndexError):
                    continue

        print(f"✓ Parsed {len(self.atoms)} atoms from PDB file")

    def setup_realtime_writers(self, output_folder: str, timestamp: str) -> None:
        """Setup CSV writers for real-time data capture."""

        # Setup file paths
        files_config = {
            'summary': {
                'path': os.path.join(output_folder, f"charge_analysis_summary_{timestamp}.csv"),
                'fieldnames': [
                    'protein_id', 'filename', 'analysis_timestamp', 'total_atoms', 'total_residues',
                    'total_chains', 'sequence_length', 'longest_chain_sequence',
                    'total_charged_residues', 'positive_residues', 'negative_residues',
                    'net_protein_charge', 'total_molecular_charge',
                    'total_binding_sites', 'strong_binding_sites', 'moderate_binding_sites',
                    'best_binding_potential', 'total_interactions', 'strong_interactions',
                    'arg_count', 'lys_count', 'his_count', 'asp_count', 'glu_count',
                    'chain_sequences'  # JSON string of all chain sequences
                ]
            },
            'sequences': {
                'path': os.path.join(output_folder, f"protein_sequences_{timestamp}.csv"),
                'fieldnames': [
                    'protein_id', 'chain_id', 'sequence_length', 'sequence',
                    'seqres_sequence', 'atom_based_sequence', 'analysis_timestamp'
                ]
            },
            'binding_sites': {
                'path': os.path.join(output_folder, f"binding_sites_{timestamp}.csv"),
                'fieldnames': [
                    'protein_id', 'site_rank', 'x_coord', 'y_coord', 'z_coord',
                    'potential', 'binding_type', 'site_size', 'analysis_timestamp'
                ]
            },
            'interactions': {
                'path': os.path.join(output_folder, f"charge_interactions_{timestamp}.csv"),
                'fieldnames': [
                    'protein_id', 'residue1', 'residue2', 'distance', 'energy',
                    'interaction_type', 'strength', 'analysis_timestamp'
                ]
            },
            'progress': {
                'path': os.path.join(output_folder, f"analysis_progress_{timestamp}.csv"),
                'fieldnames': [
                    'protein_id', 'filename', 'status', 'timestamp', 'error_message',
                    'processing_time_seconds'
                ]
            }
        }

        # Open files and create writers
        for file_type, config in files_config.items():
            try:
                file_handle = open(config['path'], 'w', newline='', encoding='utf-8')
                writer = csv.DictWriter(file_handle, fieldnames=config['fieldnames'])
                writer.writeheader()
                file_handle.flush()  # Ensure header is written immediately

                self.output_files[file_type] = file_handle
                self.output_writers[file_type] = writer

                print(f"✓ Setup real-time writer: {config['path']}")

            except Exception as e:
                print(f"✗ Failed to setup writer for {file_type}: {e}")

    def setup_realtime_writers_resume(self, output_folder: str, timestamp: str, append: bool = False) -> None:
        """Setup CSV writers for real-time data capture with resume capability."""

        # Setup file paths (same as before)
        files_config = {
            'summary': {
                'path': os.path.join(output_folder, f"charge_analysis_summary_{timestamp}.csv"),
                'fieldnames': [
                    'protein_id', 'filename', 'analysis_timestamp', 'total_atoms', 'total_residues',
                    'total_chains', 'sequence_length', 'longest_chain_sequence',
                    'total_charged_residues', 'positive_residues', 'negative_residues',
                    'net_protein_charge', 'total_molecular_charge',
                    'total_binding_sites', 'strong_binding_sites', 'moderate_binding_sites',
                    'best_binding_potential', 'total_interactions', 'strong_interactions',
                    'arg_count', 'lys_count', 'his_count', 'asp_count', 'glu_count',
                    'chain_sequences'
                ]
            },
            'sequences': {
                'path': os.path.join(output_folder, f"protein_sequences_{timestamp}.csv"),
                'fieldnames': [
                    'protein_id', 'chain_id', 'sequence_length', 'sequence',
                    'seqres_sequence', 'atom_based_sequence', 'analysis_timestamp'
                ]
            },
            'binding_sites': {
                'path': os.path.join(output_folder, f"binding_sites_{timestamp}.csv"),
                'fieldnames': [
                    'protein_id', 'site_rank', 'x_coord', 'y_coord', 'z_coord',
                    'potential', 'binding_type', 'site_size', 'analysis_timestamp'
                ]
            },
            'interactions': {
                'path': os.path.join(output_folder, f"charge_interactions_{timestamp}.csv"),
                'fieldnames': [
                    'protein_id', 'residue1', 'residue2', 'distance', 'energy',
                    'interaction_type', 'strength', 'analysis_timestamp'
                ]
            },
            'progress': {
                'path': os.path.join(output_folder, f"analysis_progress_{timestamp}.csv"),
                'fieldnames': [
                    'protein_id', 'filename', 'status', 'timestamp', 'error_message',
                    'processing_time_seconds'
                ]
            }
        }

        # Open files and create writers
        for file_type, config in files_config.items():
            try:
                # Check if file exists for append mode
                file_exists = os.path.exists(config['path'])
                mode = 'a' if (append and file_exists) else 'w'

                file_handle = open(config['path'], mode, newline='', encoding='utf-8')
                writer = csv.DictWriter(file_handle, fieldnames=config['fieldnames'])

                # Only write header if creating new file
                if mode == 'w':
                    writer.writeheader()

                file_handle.flush()

                self.output_files[file_type] = file_handle
                self.output_writers[file_type] = writer

                status = "appending to" if mode == 'a' else "created"
                print(f"✓ Setup real-time writer ({status}): {config['path']}")

            except Exception as e:
                print(f"✗ Failed to setup writer for {file_type}: {e}")

    def write_progress_update(self, protein_id: str, filename: str, status: str,
                             error_message: str = "", processing_time: float = 0.0) -> None:
        """Write progress update in real-time."""
        if 'progress' not in self.output_writers:
            return

        try:
            with self.write_lock:
                row = {
                    'protein_id': protein_id,
                    'filename': filename,
                    'status': status,
                    'timestamp': datetime.now().isoformat(),
                    'error_message': error_message,
                    'processing_time_seconds': round(processing_time, 2)
                }
                self.output_writers['progress'].writerow(row)
                self.output_files['progress'].flush()
        except Exception as e:
            print(f"⚠️ Failed to write progress update: {e}")

    def write_sequence_data(self, protein_id: str) -> None:
        """Write sequence data in real-time."""
        if 'sequences' not in self.output_writers or not self.sequence_data:
            return

        try:
            with self.write_lock:
                timestamp = datetime.now().isoformat()
                for chain_id, seq_data in self.sequence_data.items():
                    row = {
                        'protein_id': protein_id,
                        'chain_id': chain_id,
                        'sequence_length': seq_data['length'],
                        'sequence': seq_data['final_sequence'],
                        'seqres_sequence': seq_data['seqres'],
                        'atom_based_sequence': seq_data['atom_based'],
                        'analysis_timestamp': timestamp
                    }
                    self.output_writers['sequences'].writerow(row)
                self.output_files['sequences'].flush()
        except Exception as e:
            print(f"⚠️ Failed to write sequence data: {e}")

    def write_binding_sites_data(self, protein_id: str) -> None:
        """Write binding sites data in real-time."""
        if 'binding_sites' not in self.output_writers or not self.binding_sites:
            return

        try:
            with self.write_lock:
                timestamp = datetime.now().isoformat()
                # Sort binding sites by potential (best first)
                sorted_sites = sorted(self.binding_sites, key=lambda x: x['potential'])

                for i, site in enumerate(sorted_sites, 1):
                    row = {
                        'protein_id': protein_id,
                        'site_rank': i,
                        'x_coord': round(site['position'][0], 2),
                        'y_coord': round(site['position'][1], 2),
                        'z_coord': round(site['position'][2], 2),
                        'potential': round(site['potential'], 2),
                        'binding_type': site.get('type', 'unknown'),
                        'site_size': site.get('size', 1),
                        'analysis_timestamp': timestamp
                    }
                    self.output_writers['binding_sites'].writerow(row)
                self.output_files['binding_sites'].flush()
        except Exception as e:
            print(f"⚠️ Failed to write binding sites data: {e}")

    def write_interactions_data(self, protein_id: str, interactions: Dict) -> None:
        """Write interactions data in real-time."""
        if 'interactions' not in self.output_writers:
            return

        try:
            with self.write_lock:
                timestamp = datetime.now().isoformat()
                for interaction in interactions.get('interactions', []):
                    row = {
                        'protein_id': protein_id,
                        'residue1': interaction['residue1'],
                        'residue2': interaction['residue2'],
                        'distance': round(interaction['distance'], 2),
                        'energy': round(interaction['energy'], 4),
                        'interaction_type': interaction['type'],
                        'strength': interaction['strength'],
                        'analysis_timestamp': timestamp
                    }
                    self.output_writers['interactions'].writerow(row)
                self.output_files['interactions'].flush()
        except Exception as e:
            print(f"⚠️ Failed to write interactions data: {e}")

    def write_summary_data(self, protein_summary: Dict) -> None:
        """Write summary data in real-time."""
        if 'summary' not in self.output_writers:
            return

        try:
            with self.write_lock:
                # Prepare sequence data for summary
                chain_sequences = {}
                total_sequence_length = 0
                longest_sequence = ""

                for chain_id, seq_data in self.sequence_data.items():
                    chain_sequences[chain_id] = seq_data['final_sequence']
                    total_sequence_length += seq_data['length']
                    if len(seq_data['final_sequence']) > len(longest_sequence):
                        longest_sequence = seq_data['final_sequence']

                row = {
                    'protein_id': protein_summary['protein_id'],
                    'filename': protein_summary['filename'],
                    'analysis_timestamp': protein_summary['analysis_timestamp'],
                    'total_atoms': protein_summary['total_atoms'],
                    'total_residues': protein_summary['total_residues'],
                    'total_chains': protein_summary['total_chains'],
                    'sequence_length': total_sequence_length,
                    'longest_chain_sequence': longest_sequence,
                    'total_charged_residues': protein_summary['total_charged_residues'],
                    'positive_residues': protein_summary['positive_residues'],
                    'negative_residues': protein_summary['negative_residues'],
                    'net_protein_charge': round(protein_summary['net_protein_charge'], 2),
                    'total_molecular_charge': round(protein_summary['total_molecular_charge'], 2),
                    'total_binding_sites': protein_summary['total_binding_sites'],
                    'strong_binding_sites': protein_summary['strong_binding_sites'],
                    'moderate_binding_sites': protein_summary['moderate_binding_sites'],
                    'best_binding_potential': round(protein_summary['best_binding_potential'], 2),
                    'total_interactions': protein_summary['total_interactions'],
                    'strong_interactions': protein_summary['strong_interactions'],
                    'arg_count': protein_summary['positive_residue_types'].get('ARG', 0),
                    'lys_count': protein_summary['positive_residue_types'].get('LYS', 0),
                    'his_count': protein_summary['positive_residue_types'].get('HIS', 0),
                    'asp_count': protein_summary['negative_residue_types'].get('ASP', 0),
                    'glu_count': protein_summary['negative_residue_types'].get('GLU', 0),
                    'chain_sequences': json.dumps(chain_sequences)
                }
                self.output_writers['summary'].writerow(row)
                self.output_files['summary'].flush()
        except Exception as e:
            print(f"⚠️ Failed to write summary data: {e}")

    def close_realtime_writers(self) -> None:
        """Close all real-time writers."""
        with self.write_lock:
            for file_type, file_handle in self.output_files.items():
                try:
                    file_handle.close()
                    print(f"✓ Closed writer: {file_type}")
                except Exception as e:
                    print(f"⚠️ Error closing {file_type}: {e}")

            self.output_files.clear()
            self.output_writers.clear()

    def assign_partial_charges(self) -> None:
        """Assign partial charges to atoms based on residue type and atom name."""

        # Standard amino acid partial charges (AMBER ff14SB-based)
        charge_library = {
            # Charged residues
            'ARG': {'N': -0.47, 'CA': 0.26, 'C': 0.73, 'O': -0.59, 'CB': -0.00, 'CG': 0.04, 'CD': 0.05, 'NE': -0.52, 'CZ': 0.80, 'NH1': -0.86, 'NH2': -0.86},
            'LYS': {'N': -0.40, 'CA': 0.25, 'C': 0.73, 'O': -0.59, 'CB': -0.01, 'CG': 0.01, 'CD': -0.04, 'CE': -0.01, 'NZ': -0.38},
            'ASP': {'N': -0.42, 'CA': 0.27, 'C': 0.73, 'O': -0.59, 'CB': -0.16, 'CG': 0.81, 'OD1': -0.76, 'OD2': -0.76},
            'GLU': {'N': -0.42, 'CA': 0.27, 'C': 0.73, 'O': -0.59, 'CB': -0.01, 'CG': 0.01, 'CD': 0.81, 'OE1': -0.76, 'OE2': -0.76},
            'HIS': {'N': -0.42, 'CA': 0.27, 'C': 0.73, 'O': -0.59, 'CB': -0.09, 'CG': -0.05, 'ND1': -0.61, 'CD2': 0.13, 'CE1': 0.44, 'NE2': -0.61},

            # Polar residues
            'SER': {'N': -0.42, 'CA': 0.27, 'C': 0.73, 'O': -0.59, 'CB': 0.21, 'OG': -0.65},
            'THR': {'N': -0.42, 'CA': 0.27, 'C': 0.73, 'O': -0.59, 'CB': 0.20, 'OG1': -0.68, 'CG2': -0.24},
            'TYR': {'N': -0.42, 'CA': 0.27, 'C': 0.73, 'O': -0.59, 'CB': -0.01, 'CG': -0.19, 'CD1': -0.17, 'CD2': -0.17, 'CE1': -0.17, 'CE2': -0.17, 'CZ': 0.17, 'OH': -0.54},
            'CYS': {'N': -0.42, 'CA': 0.27, 'C': 0.73, 'O': -0.59, 'CB': -0.11, 'SG': -0.31},
            'ASN': {'N': -0.42, 'CA': 0.27, 'C': 0.73, 'O': -0.59, 'CB': -0.20, 'CG': 0.71, 'OD1': -0.59, 'ND2': -0.92},
            'GLN': {'N': -0.42, 'CA': 0.27, 'C': 0.73, 'O': -0.59, 'CB': -0.01, 'CG': 0.01, 'CD': 0.71, 'OE1': -0.59, 'NE2': -0.92},

            # Default for other residues (approximate)
            'DEFAULT': {'N': -0.40, 'CA': 0.25, 'C': 0.70, 'O': -0.60, 'CB': -0.05}
        }

        # Assign charges
        for atom in self.atoms:
            residue = atom['residue']
            atom_name = atom['name']

            if residue in charge_library:
                atom['charge'] = charge_library[residue].get(atom_name, 0.0)
            else:
                atom['charge'] = charge_library['DEFAULT'].get(atom_name, 0.0)

            # Special handling for metals and ions
            if atom['element'] in ['MG', 'CA', 'ZN', 'FE', 'MN']:
                atom['charge'] = 2.0 if atom['element'] in ['MG', 'CA', 'ZN'] else 3.0
            elif atom['element'] in ['CL']:
                atom['charge'] = -1.0
            elif atom['element'] in ['NA']:
                atom['charge'] = 1.0

        print(f"✓ Assigned partial charges to {len(self.atoms)} atoms")

    def identify_charged_residues(self) -> None:
        """Identify and analyze charged residues."""
        charged_types = ['ARG', 'LYS', 'ASP', 'GLU', 'HIS']
        self.charged_residues = []

        residue_groups = defaultdict(list)
        for atom in self.atoms:
            if atom['residue'] in charged_types:
                key = (atom['chain'], atom['residue'], atom['residue_id'])
                residue_groups[key].append(atom)

        for (chain, residue_type, residue_id), atoms in residue_groups.items():
            # Calculate center of mass
            coords = np.array([[a['x'], a['y'], a['z']] for a in atoms])
            center = np.mean(coords, axis=0)

            # Calculate net charge
            net_charge = sum(atom.get('charge', 0.0) for atom in atoms)

            # Determine charge type
            if residue_type in ['ARG', 'LYS']:
                charge_type = 'positive'
            elif residue_type in ['ASP', 'GLU']:
                charge_type = 'negative'
            else:  # HIS
                charge_type = 'variable'

            self.charged_residues.append({
                'chain': chain,
                'residue': residue_type,
                'residue_id': residue_id,
                'center': center,
                'atoms': atoms,
                'net_charge': net_charge,
                'charge_type': charge_type
            })

        print(f"✓ Identified {len(self.charged_residues)} charged residues")

    def find_binding_pockets(self, grid_spacing: float = 2.0, pocket_threshold: float = -5.0) -> None:
        """Find potential binding pockets based on electrostatic potential."""
        if not hasattr(self.atoms[0], 'charge') or 'charge' not in self.atoms[0]:
            print("⚠️  No charges assigned. Running charge assignment first...")
            self.assign_partial_charges()

        # Create a grid around the protein
        coords = np.array([[atom['x'], atom['y'], atom['z']] for atom in self.atoms])
        min_coords = np.min(coords, axis=0) - 5.0
        max_coords = np.max(coords, axis=0) + 5.0

        # Sample grid points
        x_range = np.arange(min_coords[0], max_coords[0], grid_spacing)
        y_range = np.arange(min_coords[1], max_coords[1], grid_spacing)
        z_range = np.arange(min_coords[2], max_coords[2], grid_spacing)

        binding_sites = []
        print(f"🔍 Scanning {len(x_range) * len(y_range) * len(z_range)} grid points...")

        # Sample fewer points for performance
        x_sample = x_range[::2]  # Every other point
        y_sample = y_range[::2]
        z_sample = z_range[::2]

        for i, x in enumerate(x_sample):
            if i % 10 == 0:
                print(f"  Progress: {i}/{len(x_sample)} ({100*i/len(x_sample):.0f}%)")

            for y in y_sample:
                for z in z_sample:
                    grid_point = np.array([x, y, z])

                    # Check if point is not too close to any atom (avoid surface)
                    distances = np.sqrt(np.sum((coords - grid_point)**2, axis=1))
                    min_distance = np.min(distances)

                    if min_distance < 2.0 or min_distance > 8.0:  # Skip points too close or too far
                        continue

                    # Calculate electrostatic potential at this point
                    potential = 0.0
                    for atom in self.atoms:
                        atom_pos = np.array([atom['x'], atom['y'], atom['z']])
                        distance = np.linalg.norm(grid_point - atom_pos)
                        if distance > 0.1:  # Avoid division by zero
                            potential += atom.get('charge', 0.0) / distance

                    # Check if this is a favorable binding site
                    if potential < pocket_threshold:
                        binding_sites.append({
                            'position': grid_point,
                            'potential': potential,
                            'type': 'cation_binding' if potential < -10 else 'weak_binding'
                        })

        # Cluster nearby binding sites
        self.binding_sites = self.cluster_binding_sites(binding_sites)
        print(f"✓ Found {len(self.binding_sites)} potential binding sites")

    def cluster_binding_sites(self, sites: List[Dict], cluster_radius: float = 3.0) -> List[Dict]:
        """Cluster nearby binding sites into single sites."""
        if not sites:
            return []

        positions = np.array([site['position'] for site in sites])
        potentials = np.array([site['potential'] for site in sites])

        clustered_sites = []
        used = np.zeros(len(sites), dtype=bool)

        for i, site in enumerate(sites):
            if used[i]:
                continue

            # Find nearby sites
            distances = np.sqrt(np.sum((positions - positions[i])**2, axis=1))
            cluster_mask = distances < cluster_radius

            # Calculate cluster center and average potential
            cluster_positions = positions[cluster_mask]
            cluster_potentials = potentials[cluster_mask]

            center = np.mean(cluster_positions, axis=0)
            avg_potential = np.mean(cluster_potentials)
            min_potential = np.min(cluster_potentials)

            clustered_sites.append({
                'position': center,
                'potential': avg_potential,
                'min_potential': min_potential,
                'size': np.sum(cluster_mask),
                'type': 'strong_binding' if min_potential < -15 else 'moderate_binding' if min_potential < -8 else 'weak_binding'
            })

            used[cluster_mask] = True

        return clustered_sites

    def analyze_charge_interactions(self) -> Dict:
        """Analyze electrostatic interactions between charged residues."""
        if not self.charged_residues:
            self.identify_charged_residues()

        interactions = []

        for i, res1 in enumerate(self.charged_residues):
            for res2 in self.charged_residues[i+1:]:
                distance = np.linalg.norm(res1['center'] - res2['center'])

                # Calculate interaction energy (simplified Coulomb)
                q1 = res1['net_charge']
                q2 = res2['net_charge']
                energy = (q1 * q2) / distance if distance > 0 else 0

                interaction_type = 'attractive' if energy < 0 else 'repulsive'
                strength = 'strong' if abs(energy) > 0.5 else 'moderate' if abs(energy) > 0.1 else 'weak'

                interactions.append({
                    'residue1': f"{res1['chain']}-{res1['residue']}{res1['residue_id']}",
                    'residue2': f"{res2['chain']}-{res2['residue']}{res2['residue_id']}",
                    'distance': distance,
                    'energy': energy,
                    'type': interaction_type,
                    'strength': strength
                })

        return {
            'interactions': interactions,
            'total_positive': sum(1 for r in self.charged_residues if r['charge_type'] == 'positive'),
            'total_negative': sum(1 for r in self.charged_residues if r['charge_type'] == 'negative'),
            'net_charge': sum(r['net_charge'] for r in self.charged_residues)
        }

    def get_statistics(self) -> Dict:
        """Get basic statistics about the structure."""
        if not self.atoms:
            return {}

        elements = [atom['element'] for atom in self.atoms]
        residues = [atom['residue'] for atom in self.atoms]
        chains = [atom['chain'] for atom in self.atoms]

        coords = np.array([[atom['x'], atom['y'], atom['z']] for atom in self.atoms])

        stats = {
            'total_atoms': len(self.atoms),
            'unique_elements': len(set(elements)),
            'unique_residues': len(set(residues)),
            'unique_chains': len(set(chains)),
            'element_counts': {elem: elements.count(elem) for elem in set(elements)},
            'center': np.mean(coords, axis=0),
            'size': np.max(coords, axis=0) - np.min(coords, axis=0)
        }

        return stats

    def compile_protein_summary(self, protein_id: str, interactions: Dict) -> Dict:
        """Compile comprehensive summary for a single protein."""
        stats = self.get_statistics() if self.atoms else {}

        # Binding site analysis
        strong_sites = [s for s in self.binding_sites if s.get('type') == 'strong_binding']
        moderate_sites = [s for s in self.binding_sites if s.get('type') == 'moderate_binding']

        # Charge distribution
        positive_residues = [r for r in self.charged_residues if r['charge_type'] == 'positive']
        negative_residues = [r for r in self.charged_residues if r['charge_type'] == 'negative']

        # Strong interactions
        strong_interactions = [i for i in interactions.get('interactions', []) if i['strength'] == 'strong']

        # Sequence information
        total_sequence_length = sum(data['length'] for data in self.sequence_data.values())

        return {
            'protein_id': protein_id,
            'filename': self.filename,
            'analysis_timestamp': datetime.now().isoformat(),

            # Basic structure info
            'total_atoms': len(self.atoms),
            'total_residues': stats.get('unique_residues', 0),
            'total_chains': stats.get('unique_chains', 0),
            'sequence_length': total_sequence_length,
            'sequence_data': self.sequence_data,

            # Charge analysis
            'total_charged_residues': len(self.charged_residues),
            'positive_residues': len(positive_residues),
            'negative_residues': len(negative_residues),
            'net_protein_charge': sum(r['net_charge'] for r in self.charged_residues),
            'total_molecular_charge': sum(atom.get('charge', 0.0) for atom in self.atoms),

            # Binding sites
            'total_binding_sites': len(self.binding_sites),
            'strong_binding_sites': len(strong_sites),
            'moderate_binding_sites': len(moderate_sites),
            'best_binding_potential': min([s['potential'] for s in self.binding_sites]) if self.binding_sites else 0,

            # Interactions
            'total_interactions': len(interactions.get('interactions', [])),
            'strong_interactions': len(strong_interactions),

            # Detailed residue breakdown
            'positive_residue_types': {res_type: len([r for r in positive_residues if r['residue'] == res_type])
                                    for res_type in ['ARG', 'LYS', 'HIS']},
            'negative_residue_types': {res_type: len([r for r in negative_residues if r['residue'] == res_type])
                                     for res_type in ['ASP', 'GLU']},

            # Top binding sites (up to 5)
            'top_binding_sites': sorted(self.binding_sites, key=lambda x: x['potential'])[:5] if self.binding_sites else [],

            # Key interactions
            'key_interactions': strong_interactions[:10] if strong_interactions else [],

            # Charge distribution stats
            'charge_stats': {
                'min_charge': min([atom.get('charge', 0.0) for atom in self.atoms]) if self.atoms else 0,
                'max_charge': max([atom.get('charge', 0.0) for atom in self.atoms]) if self.atoms else 0,
                'charge_std': np.std([atom.get('charge', 0.0) for atom in self.atoms]) if self.atoms else 0
            }
        }

    def batch_analyze_folder_realtime_with_resume(self, folder_path: str, output_folder: str = None, resume: bool = True) -> Dict:
        """
        Enhanced batch analyze with RESUME capability - picks up where it left off.

        Args:
            folder_path: Path to folder containing .pdb.gz files
            output_folder: Path to output folder (default: folder_path/charge_analysis_results)
            resume: If True, attempts to resume from previous run

        Returns:
            Dictionary with summary statistics across all analyzed proteins
        """

        if not os.path.exists(folder_path):
            print(f"✗ Folder not found: {folder_path}")
            return {}

        # Set up output directory
        if output_folder is None:
            output_folder = os.path.join(folder_path, "charge_analysis_results")

        os.makedirs(output_folder, exist_ok=True)

        # Find all .pdb.gz files
        pdb_files = glob.glob(os.path.join(folder_path, "*.pdb.gz"))
        pdb_files.extend(glob.glob(os.path.join(folder_path, "*.pdb")))

        if not pdb_files:
            print(f"✗ No .pdb or .pdb.gz files found in {folder_path}")
            return {}

        print(f"🔍 Found {len(pdb_files)} PDB files to analyze")

        # ═══════════════════════════════════════════════════════
        # RESUME LOGIC - Check for existing progress
        # ═══════════════════════════════════════════════════════
        completed_proteins = set()
        failed_proteins = set()
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        existing_timestamp = None

        if resume:
            # Look for existing progress files
            existing_progress_files = glob.glob(os.path.join(output_folder, "analysis_progress_*.csv"))

            if existing_progress_files:
                # Find the most recent progress file
                latest_progress_file = max(existing_progress_files, key=os.path.getmtime)
                existing_timestamp = latest_progress_file.split('_')[-1].replace('.csv', '')

                print(f"📋 Found existing progress file: {os.path.basename(latest_progress_file)}")

                try:
                    import pandas as pd
                    progress_df = pd.read_csv(latest_progress_file)

                    completed_proteins = set(progress_df[progress_df['status'] == 'COMPLETED']['protein_id'].tolist())
                    failed_proteins = set(progress_df[progress_df['status'] == 'FAILED']['protein_id'].tolist())

                    print(f"✅ Previously completed: {len(completed_proteins)} proteins")
                    print(f"❌ Previously failed: {len(failed_proteins)} proteins")

                    if completed_proteins:
                        print(f"📄 Last completed: {list(completed_proteins)[-5:]}")

                    # Use existing timestamp to continue same batch
                    timestamp = existing_timestamp
                    print(f"🔄 Resuming batch with timestamp: {timestamp}")

                except Exception as e:
                    print(f"⚠️ Error reading progress file: {e}")
                    print("🔄 Starting fresh analysis")
                    resume = False

        # Filter out already completed proteins
        remaining_files = []
        skipped_completed = 0
        skipped_failed = 0

        for filepath in pdb_files:
            filename = os.path.basename(filepath)
            protein_id = filename.split('.')[0].upper()

            if protein_id in completed_proteins:
                skipped_completed += 1
                continue
            elif protein_id in failed_proteins:
                print(f"⚠️ Skipping previously failed protein: {protein_id}")
                skipped_failed += 1
                continue
            else:
                remaining_files.append(filepath)

        print(f"📊 Analysis Summary:")
        print(f"  📁 Total files found: {len(pdb_files)}")
        print(f"  ✅ Already completed: {skipped_completed}")
        print(f"  ❌ Previously failed: {skipped_failed}")
        print(f"  🔄 Remaining to process: {len(remaining_files)}")

        if not remaining_files:
            print("🎉 All proteins already completed!")
            return {
                'analyzed': len(completed_proteins),
                'failed': len(failed_proteins),
                'output_folder': output_folder,
                'results': [],
                'already_completed': True
            }

        print(f"📁 Output will be saved to: {output_folder}")

        # Setup real-time writers (will append to existing files if resuming)
        if resume and existing_timestamp:
            # Use existing timestamp and append mode
            self.setup_realtime_writers_resume(output_folder, timestamp, append=True)
        else:
            # Create new files
            self.setup_realtime_writers(output_folder, timestamp)

        # Initialize batch results
        batch_results = []
        failed_files = []

        try:
            # Process remaining files
            for i, filepath in enumerate(remaining_files):
                filename = os.path.basename(filepath)
                protein_id = filename.split('.')[0].upper()

                print(f"\n📊 Processing {i+1}/{len(remaining_files)}: {protein_id} (Overall: {skipped_completed + skipped_failed + i + 1}/{len(pdb_files)})")
                start_time = time.time()

                try:
                    # Write initial progress
                    self.write_progress_update(protein_id, filename, "STARTED")

                    # Reset analyzer for new structure
                    self.atoms = []
                    self.charged_residues = []
                    self.binding_sites = []
                    self.sequence_data = {}

                    # Load and parse structure
                    content = self.open_file(filepath)
                    if not content:
                        error_msg = "Failed to read file"
                        failed_files.append((filename, error_msg))
                        self.write_progress_update(protein_id, filename, "FAILED", error_msg)
                        continue

                    if filepath.lower().endswith(('.pdb.gz', '.pdb')):
                        self.parse_pdb(content)
                    else:
                        error_msg = "Unsupported format"
                        failed_files.append((filename, error_msg))
                        self.write_progress_update(protein_id, filename, "FAILED", error_msg)
                        continue

                    if not self.atoms:
                        error_msg = "No atoms parsed"
                        failed_files.append((filename, error_msg))
                        self.write_progress_update(protein_id, filename, "FAILED", error_msg)
                        continue

                    # Write sequence data immediately after parsing
                    self.write_sequence_data(protein_id)
                    self.write_progress_update(protein_id, filename, "SEQUENCES_EXTRACTED")

                    # Perform charge analysis
                    print("  ⚡ Assigning charges...")
                    self.assign_partial_charges()
                    self.write_progress_update(protein_id, filename, "CHARGES_ASSIGNED")

                    print("  🔍 Identifying charged residues...")
                    self.identify_charged_residues()
                    self.write_progress_update(protein_id, filename, "CHARGED_RESIDUES_IDENTIFIED")

                    print("  🔗 Analyzing interactions...")
                    interactions = self.analyze_charge_interactions()
                    self.write_interactions_data(protein_id, interactions)
                    self.write_progress_update(protein_id, filename, "INTERACTIONS_ANALYZED")

                    # Calculate binding pockets (quick version)
                    print("  🎯 Finding binding sites...")
                    self.find_binding_pockets(grid_spacing=3.0, pocket_threshold=-3.0)
                    self.write_binding_sites_data(protein_id)
                    self.write_progress_update(protein_id, filename, "BINDING_SITES_FOUND")

                    # Compile and write summary
                    protein_result = self.compile_protein_summary(protein_id, interactions)
                    self.write_summary_data(protein_result)
                    batch_results.append(protein_result)

                    # Final progress update
                    processing_time = time.time() - start_time
                    self.write_progress_update(protein_id, filename, "COMPLETED", "", processing_time)

                    print(f"  ✅ {protein_id}: {len(self.atoms)} atoms, {len(self.charged_residues)} charged residues, "
                          f"{len(self.binding_sites)} binding sites, {sum(len(seq['final_sequence']) for seq in self.sequence_data.values())} total sequence length")

                except Exception as e:
                    processing_time = time.time() - start_time
                    error_msg = f"Analysis failed: {str(e)}"
                    failed_files.append((filename, error_msg))
                    self.write_progress_update(protein_id, filename, "FAILED", error_msg, processing_time)
                    print(f"  ✗ {protein_id}: {error_msg}")
                    continue

        finally:
            # Always close writers, even if there's an error
            self.close_realtime_writers()

        # Generate final consolidated reports (only if new proteins were processed)
        if batch_results:
            json_file = os.path.join(output_folder, f"charge_analysis_detailed_{timestamp}.json")
            self.export_final_json_report(batch_results, failed_files, json_file)

            report_file = os.path.join(output_folder, f"charge_analysis_report_{timestamp}.txt")
            self.export_final_text_report(batch_results, failed_files, report_file)

        total_analyzed = len(completed_proteins) + len(batch_results)
        total_failed = len(failed_proteins) + len(failed_files)

        print(f"\n🎉 Batch analysis complete!")
        print(f"  ✅ Total analyzed: {total_analyzed} proteins ({len(batch_results)} new)")
        print(f"  ✗ Total failed: {total_failed} proteins ({len(failed_files)} new)")
        print(f"  📁 Results saved to: {output_folder}")
        if resume and skipped_completed > 0:
            print(f"  ⏭️  Skipped {skipped_completed} already completed proteins")

        return {
            'analyzed': total_analyzed,
            'failed': total_failed,
            'new_analyzed': len(batch_results),
            'new_failed': len(failed_files),
            'skipped_completed': skipped_completed,
            'skipped_failed': skipped_failed,
            'output_folder': output_folder,
            'results': batch_results,
            'realtime_files': {
                'summary': f"charge_analysis_summary_{timestamp}.csv",
                'sequences': f"protein_sequences_{timestamp}.csv",
                'binding_sites': f"binding_sites_{timestamp}.csv",
                'interactions': f"charge_interactions_{timestamp}.csv",
                'progress': f"analysis_progress_{timestamp}.csv"
            }
        }

    def export_final_json_report(self, results: List[Dict], failed_files: List, filepath: str) -> None:
        """Export detailed JSON report with all analysis data including sequences."""
        report_data = {
            'analysis_metadata': {
                'timestamp': datetime.now().isoformat(),
                'total_proteins_analyzed': len(results),
                'total_failed': len(failed_files),
                'analysis_version': '2.0_with_sequences_and_resume',
                'features': [
                    'real_time_data_capture',
                    'sequence_extraction',
                    'electrostatic_analysis',
                    'binding_site_prediction',
                    'charge_interaction_analysis',
                    'resume_capability'
                ]
            },
            'failed_files': [{'filename': f[0], 'error': f[1]} for f in failed_files],
            'protein_analyses': results,
            'summary_statistics': self.calculate_batch_statistics(results) if results else {}
        }

        with open(filepath, 'w') as jsonfile:
            json.dump(report_data, jsonfile, indent=2, default=str)

        print(f"  📋 Final detailed JSON saved: {filepath}")

    def export_final_text_report(self, results: List[Dict], failed_files: List, filepath: str) -> None:
        """Export enhanced human-readable text report with sequence information."""
        with open(filepath, 'w') as f:
            f.write("ENHANCED PROTEIN CHARGE AFFINITY ANALYSIS REPORT (WITH RESUME)\n")
            f.write("=" * 70 + "\n\n")
            f.write(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Total Proteins Analyzed: {len(results)}\n")
            f.write(f"Failed Analyses: {len(failed_files)}\n")
            f.write("Features: Real-time data capture, Sequence extraction, Resume capability\n\n")

            if failed_files:
                f.write("FAILED ANALYSES:\n")
                f.write("-" * 20 + "\n")
                for filename, error in failed_files:
                    f.write(f"  {filename}: {error}\n")
                f.write("\n")

            f.write("SUMMARY STATISTICS:\n")
            f.write("-" * 20 + "\n")
            if results:
                stats = self.calculate_batch_statistics(results)
                f.write(f"  Average net charge: {stats['avg_net_charge']:.2f}\n")
                f.write(f"  Average sequence length: {stats['avg_sequence_length']:.1f}\n")
                f.write(f"  Average binding sites: {stats['avg_binding_sites']:.1f}\n")
                f.write(f"  Proteins with strong binding sites: {stats['proteins_with_strong_sites']}\n")
                f.write(f"  Total sequence length analyzed: {stats['total_sequence_length']}\n")
                f.write(f"  Most common charged residues: {stats['most_common_charged_residues']}\n\n")

            f.write("INDIVIDUAL PROTEIN RESULTS:\n")
            f.write("-" * 30 + "\n")
            for result in results:
                f.write(f"\nProtein ID: {result['protein_id']}\n")
                f.write(f"  Filename: {result['filename']}\n")
                f.write(f"  Total atoms: {result['total_atoms']}\n")
                f.write(f"  Total chains: {result['total_chains']}\n")
                f.write(f"  Sequence length: {result['sequence_length']} residues\n")
                f.write(f"  Charged residues: {result['total_charged_residues']} (+{result['positive_residues']}/-{result['negative_residues']})\n")
                f.write(f"  Net charge: {result['net_protein_charge']:.2f}\n")
                f.write(f"  Binding sites: {result['total_binding_sites']} ({result['strong_binding_sites']} strong)\n")
                f.write(f"  Strong interactions: {result['strong_interactions']}\n")

                # Sequence information per chain
                if result['sequence_data']:
                    f.write(f"  Chain sequences:\n")
                    for chain_id, seq_data in result['sequence_data'].items():
                        f.write(f"    Chain {chain_id}: {seq_data['length']} residues\n")

                if result['top_binding_sites']:
                    f.write(f"  Best binding potential: {result['best_binding_potential']:.2f}\n")

        print(f"  📄 Final text report saved: {filepath}")

    def calculate_batch_statistics(self, results: List[Dict]) -> Dict:
        """Calculate comprehensive statistics across all analyzed proteins."""
        if not results:
            return {}

        # Basic statistics
        net_charges = [r['net_protein_charge'] for r in results]
        sequence_lengths = [r['sequence_length'] for r in results]
        binding_sites = [r['total_binding_sites'] for r in results]

        # Charged residue counts
        all_charged_residues = []
        for result in results:
            for res_type, count in result['positive_residue_types'].items():
                all_charged_residues.extend([res_type] * count)
            for res_type, count in result['negative_residue_types'].items():
                all_charged_residues.extend([res_type] * count)

        # Count occurrences
        from collections import Counter
        residue_counts = Counter(all_charged_residues)

        return {
            'avg_net_charge': np.mean(net_charges),
            'std_net_charge': np.std(net_charges),
            'avg_sequence_length': np.mean(sequence_lengths),
            'std_sequence_length': np.std(sequence_lengths),
            'avg_binding_sites': np.mean(binding_sites),
            'proteins_with_strong_sites': len([r for r in results if r['strong_binding_sites'] > 0]),
            'total_sequence_length': sum(sequence_lengths),
            'total_proteins': len(results),
            'most_common_charged_residues': dict(residue_counts.most_common(5)),
            'charge_distribution': {
                'positive_proteins': len([r for r in results if r['net_protein_charge'] > 0]),
                'negative_proteins': len([r for r in results if r['net_protein_charge'] < 0]),
                'neutral_proteins': len([r for r in results if abs(r['net_protein_charge']) < 0.1])
            }
        }

    def visualize_charge_distribution(self) -> None:
        """Visualize charge distribution and binding sites with sequence information."""
        if not self.atoms:
            print("No structure loaded!")
            return

        if not hasattr(self.atoms[0], 'charge') or 'charge' not in self.atoms[0]:
            self.assign_partial_charges()

        if not self.charged_residues:
            self.identify_charged_residues()

        # Check if we have enough data to plot
        if len(self.atoms) == 0:
            print("⚠️  No atoms to visualize!")
            return

        try:
            fig = plt.figure(figsize=(18, 14))

            # Main 3D plot
            ax1 = fig.add_subplot(231, projection='3d')

            # Plot all atoms (faded)
            coords = np.array([[atom['x'], atom['y'], atom['z']] for atom in self.atoms])
            ax1.scatter(coords[:, 0], coords[:, 1], coords[:, 2],
                       c='lightgray', alpha=0.3, s=10, label='atoms')

            # Plot charged residues with proper legend handling
            legend_labels_added = set()
            if self.charged_residues:
                for residue in self.charged_residues:
                    color = 'red' if residue['charge_type'] == 'positive' else 'blue' if residue['charge_type'] == 'negative' else 'green'
                    size = abs(residue['net_charge']) * 100 + 50

                    # Add label only for first occurrence of each charge type
                    label = residue['charge_type'] if residue['charge_type'] not in legend_labels_added else ""
                    if label:
                        legend_labels_added.add(label)

                    ax1.scatter(residue['center'][0], residue['center'][1], residue['center'][2],
                               c=color, s=size, alpha=0.8, label=label)

            # Plot binding sites
            if self.binding_sites:
                binding_labels_added = set()
                for site in self.binding_sites:
                    color = 'yellow' if site['type'] == 'strong_binding' else 'orange'

                    label = site['type'] if site['type'] not in binding_labels_added else ""
                    if label:
                        binding_labels_added.add(label)

                    ax1.scatter(site['position'][0], site['position'][1], site['position'][2],
                               c=color, marker='s', s=100, alpha=0.7, label=label)

            ax1.set_xlabel('X (Å)')
            ax1.set_ylabel('Y (Å)')
            ax1.set_zlabel('Z (Å)')
            ax1.set_title(f'Charge Distribution & Binding Sites\n{self.filename}')

            # Add legend if we have labeled items
            handles, labels = ax1.get_legend_handles_labels()
            if labels:
                ax1.legend()

            # Charge distribution histogram
            ax2 = fig.add_subplot(232)
            charges = [atom.get('charge', 0.0) for atom in self.atoms if 'charge' in atom]
            if charges:
                ax2.hist(charges, bins=50, alpha=0.7, color='skyblue')
                ax2.set_xlabel('Partial Charge')
                ax2.set_ylabel('Number of Atoms')
                ax2.set_title('Charge Distribution')

            # Residue charge analysis
            ax3 = fig.add_subplot(233)
            if self.charged_residues:
                charge_types = [r['charge_type'] for r in self.charged_residues]
                type_counts = {t: charge_types.count(t) for t in set(charge_types)}
                colors = {'positive': 'red', 'negative': 'blue', 'variable': 'green'}
                bar_colors = [colors.get(t, 'gray') for t in type_counts.keys()]
                ax3.bar(type_counts.keys(), type_counts.values(), color=bar_colors, alpha=0.7)
                ax3.set_ylabel('Count')
                ax3.set_title('Charged Residue Types')

            # Binding site potential
            ax4 = fig.add_subplot(234)
            if self.binding_sites:
                potentials = [site['potential'] for site in self.binding_sites]
                ax4.hist(potentials, bins=20, alpha=0.7, color='orange')
                ax4.set_xlabel('Electrostatic Potential')
                ax4.set_ylabel('Number of Sites')
                ax4.set_title('Binding Site Potentials')

            # Sequence length per chain
            ax5 = fig.add_subplot(235)
            if self.sequence_data:
                chain_ids = list(self.sequence_data.keys())
                chain_lengths = [self.sequence_data[chain]['length'] for chain in chain_ids]
                ax5.bar(chain_ids, chain_lengths, alpha=0.7, color='green')
                ax5.set_xlabel('Chain ID')
                ax5.set_ylabel('Sequence Length')
                ax5.set_title('Chain Sequence Lengths')

            # Overall statistics
            ax6 = fig.add_subplot(236)
            ax6.axis('off')
            stats_text = f"Structure Statistics:\n"
            stats_text += f"Total atoms: {len(self.atoms)}\n"
            stats_text += f"Total chains: {len(self.sequence_data)}\n"
            if self.sequence_data:
                total_seq_len = sum(data['length'] for data in self.sequence_data.values())
                stats_text += f"Total sequence length: {total_seq_len}\n"
            stats_text += f"Charged residues: {len(self.charged_residues)}\n"
            stats_text += f"Binding sites: {len(self.binding_sites)}\n"

            if self.sequence_data:
                stats_text += f"\nSequence Information:\n"
                for chain_id, seq_data in self.sequence_data.items():
                    stats_text += f"Chain {chain_id}: {seq_data['length']} residues\n"

            ax6.text(0.1, 0.9, stats_text, transform=ax6.transAxes, fontsize=10,
                    verticalalignment='top', fontfamily='monospace')

            plt.tight_layout()
            plt.show()

        except Exception as e:
            print(f"⚠️  Error creating visualization: {e}")

    def print_enhanced_analysis(self) -> None:
        """Print comprehensive charge analysis with sequence information."""
        if not self.atoms:
            print("No structure loaded!")
            return

        if not hasattr(self.atoms[0], 'charge') or 'charge' not in self.atoms[0]:
            self.assign_partial_charges()

        if not self.charged_residues:
            self.identify_charged_residues()

        print(f"\n⚡ Enhanced Charge Affinity Analysis for: {self.filename}")
        print("=" * 70)

        # Sequence information
        if self.sequence_data:
            print(f"\n🧬 Sequence Information:")
            total_length = 0
            for chain_id, seq_data in self.sequence_data.items():
                print(f"  Chain {chain_id}: {seq_data['length']} residues")
                total_length += seq_data['length']
                if seq_data['length'] <= 50:  # Show short sequences
                    print(f"    Sequence: {seq_data['final_sequence']}")
            print(f"  Total sequence length: {total_length} residues")

        # Overall statistics
        total_charge = sum(atom.get('charge', 0.0) for atom in self.atoms)
        print(f"\n⚡ Charge Analysis:")
        print(f"  Total molecular charge: {total_charge:.2f}")
        print(f"  Total charged residues: {len(self.charged_residues)}")

        # Charged residue breakdown
        positive_residues = [r for r in self.charged_residues if r['charge_type'] == 'positive']
        negative_residues = [r for r in self.charged_residues if r['charge_type'] == 'negative']

        print(f"\n📊 Charged Residue Distribution:")
        print(f"  Positive residues: {len(positive_residues)} (ARG, LYS)")
        print(f"  Negative residues: {len(negative_residues)} (ASP, GLU)")
        print(f"  Net charge from residues: {sum(r['net_charge'] for r in self.charged_residues):.1f}")

        # Binding sites
        if self.binding_sites:
            print(f"\n🎯 Potential Binding Sites: {len(self.binding_sites)}")
            strong_sites = [s for s in self.binding_sites if s['type'] == 'strong_binding']
            moderate_sites = [s for s in self.binding_sites if s['type'] == 'moderate_binding']

            print(f"  Strong binding sites: {len(strong_sites)}")
            print(f"  Moderate binding sites: {len(moderate_sites)}")

            if strong_sites:
                print(f"\n🏆 Top Binding Sites:")
                sorted_sites = sorted(self.binding_sites, key=lambda x: x['potential'])
                for i, site in enumerate(sorted_sites[:5]):
                    print(f"  Site {i+1}: Potential {site['potential']:.1f} at ({site['position'][0]:.1f}, {site['position'][1]:.1f}, {site['position'][2]:.1f})")

        # Interaction analysis
        interactions = self.analyze_charge_interactions()
        strong_interactions = [i for i in interactions['interactions'] if i['strength'] == 'strong']

        print(f"\n🔗 Electrostatic Interactions:")
        print(f"  Total interactions analyzed: {len(interactions['interactions'])}")
        print(f"  Strong interactions: {len(strong_interactions)}")

        if strong_interactions:
            print(f"\n⚡ Key Interactions:")
            for interaction in strong_interactions[:5]:
                print(f"  {interaction['residue1']} ↔ {interaction['residue2']}: "
                      f"{interaction['distance']:.1f}Å, {interaction['type']}")

In [ ]:
def main():
    """Enhanced main function with real-time analysis, sequence extraction, and resume capability."""
    analyzer = EnhancedChargeAffinityAnalyzer()

    print("⚡ Enhanced Molecular Charge Affinity Analyzer v2.1")
    print("=" * 65)
    print("Features:")
    print("✓ Real-time data capture during analysis")
    print("✓ Protein sequence extraction and analysis")
    print("✓ Enhanced reporting with sequence information")
    print("✓ Progress tracking and error handling")
    print("✓ RESUME capability - picks up where it left off!")
    print()
    print("Options:")
    print("1. Analyze single file (enhanced)")
    print("2. Batch analyze folder (real-time with resume)")
    print("3. Exit")

    mode_choice = input("\nSelect mode (1-3): ").strip()

    if mode_choice == '2':
        # Enhanced batch analysis mode with resume
        folder_path = input("Enter folder path containing .pdb/.pdb.gz files: ").strip()
        if folder_path and os.path.exists(folder_path):
            output_path = input("Enter output folder (press Enter for default): ").strip()
            output_path = output_path if output_path else None

            resume_choice = input("Resume from previous run if possible? (Y/n): ").strip().lower()
            resume = resume_choice != 'n'

            if resume:
                print("\n🔄 Starting enhanced batch analysis with RESUME capability...")
            else:
                print("\n🆕 Starting fresh batch analysis...")

            batch_summary = analyzer.batch_analyze_folder_realtime_with_resume(
                folder_path, output_path, resume=resume
            )

            if batch_summary:
                print(f"\n📈 Enhanced Batch Analysis Summary:")
                print(f"  ✅ Total analyzed: {batch_summary.get('analyzed', 0)} proteins")
                print(f"  ✗ Total failed: {batch_summary.get('failed', 0)} proteins")
                if 'new_analyzed' in batch_summary:
                    print(f"  🆕 Newly analyzed: {batch_summary['new_analyzed']} proteins")
                    print(f"  ⏭️  Skipped (completed): {batch_summary.get('skipped_completed', 0)} proteins")
                print(f"  📁 Results saved to: {batch_summary['output_folder']}")

                if 'realtime_files' in batch_summary:
                    print(f"\n📊 Real-time files generated:")
                    for file_type, filename in batch_summary['realtime_files'].items():
                        print(f"  📄 {file_type.title()}: {filename}")

                # Quick stats
                if batch_summary.get('results'):
                    results = batch_summary['results']
                    if results:
                        avg_charge = np.mean([r['net_protein_charge'] for r in results])
                        total_sites = sum([r['total_binding_sites'] for r in results])
                        total_sequence_length = sum([r['sequence_length'] for r in results])

                        print(f"\n📊 Quick Statistics (new analyses):")
                        print(f"  Average net charge: {avg_charge:.2f}")
                        print(f"  Total binding sites found: {total_sites}")
                        print(f"  Total sequence length analyzed: {total_sequence_length} residues")
                        print(f"  Proteins with strong binding sites: {len([r for r in results if r['strong_binding_sites'] > 0])}")
                        if len(results) > 0:
                            print(f"  Average sequence length: {total_sequence_length/len(results):.1f} residues/protein")
        else:
            print("✗ Invalid folder path")
        return

    elif mode_choice == '3':
        print("👋 Goodbye!")
        return

    elif mode_choice != '1':
        print("✗ Invalid choice. Exiting.")
        return

    # Enhanced single file analysis mode
    filepath = input("Enter the path to your .pdb.gz or .pdb file: ").strip()

    if not os.path.exists(filepath):
        print(f"✗ File not found: {filepath}")
        return

    # Load structure
    content = analyzer.open_file(filepath)
    if not content:
        return

    # Parse structure
    if filepath.lower().endswith(('.pdb.gz', '.pdb')):
        analyzer.parse_pdb(content)
    else:
        print("✗ Currently only PDB format supported for charge analysis")
        return

    # Perform initial analysis
    print("\n🔬 Performing enhanced charge analysis...")
    analyzer.assign_partial_charges()
    analyzer.identify_charged_residues()

    # Interactive menu for single file
    while True:
        print(f"\n⚡ Enhanced Analysis Options for {analyzer.filename}:")
        print("1. Show enhanced charge analysis summary")
        print("2. Show sequence information")
        print("3. Visualize charge distribution (enhanced)")
        print("4. Find binding pockets (slow)")
        print("5. Analyze charge interactions")
        print("6. Export current analysis to CSV")
        print("7. Batch analyze folder with resume (switch to batch mode)")
        print("8. Exit")

        choice = input("\nEnter your choice (1-8): ").strip()

        if choice == '1':
            analyzer.print_enhanced_analysis()
        elif choice == '2':
            print(f"\n🧬 Sequence Information for {analyzer.filename}:")
            print("=" * 50)
            if analyzer.sequence_data:
                total_length = 0
                for chain_id, seq_data in analyzer.sequence_data.items():
                    print(f"\nChain {chain_id}:")
                    print(f"  Length: {seq_data['length']} residues")
                    print(f"  SEQRES sequence: {seq_data['seqres'][:60]}{'...' if len(seq_data['seqres']) > 60 else ''}")
                    print(f"  Atom-based sequence: {seq_data['atom_based'][:60]}{'...' if len(seq_data['atom_based']) > 60 else ''}")
                    print(f"  Final sequence: {seq_data['final_sequence'][:60]}{'...' if len(seq_data['final_sequence']) > 60 else ''}")
                    total_length += seq_data['length']
                print(f"\nTotal sequence length: {total_length} residues")
            else:
                print("No sequence data available.")
        elif choice == '3':
            analyzer.visualize_charge_distribution()
        elif choice == '4':
            print("🔍 Finding binding pockets (this may take a few minutes)...")
            analyzer.find_binding_pockets()
            analyzer.print_enhanced_analysis()
        elif choice == '5':
            interactions = analyzer.analyze_charge_interactions()
            print(f"\n🔗 Found {len(interactions['interactions'])} interactions")
            for i, interaction in enumerate(interactions['interactions'][:10]):
                print(f"{i+1}. {interaction['residue1']} ↔ {interaction['residue2']}: "
                      f"{interaction['distance']:.1f}Å ({interaction['type']}, {interaction['strength']})")
        elif choice == '6':
            # Export current analysis
            output_dir = input("Enter output directory (press Enter for current directory): ").strip()
            if not output_dir:
                output_dir = "."

            try:
                os.makedirs(output_dir, exist_ok=True)
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                protein_id = analyzer.filename.split('.')[0].upper()

                # Setup writers
                analyzer.setup_realtime_writers(output_dir, timestamp)

                # Export data
                analyzer.write_sequence_data(protein_id)

                if analyzer.binding_sites:
                    analyzer.write_binding_sites_data(protein_id)

                interactions = analyzer.analyze_charge_interactions()
                analyzer.write_interactions_data(protein_id, interactions)

                protein_summary = analyzer.compile_protein_summary(protein_id, interactions)
                analyzer.write_summary_data(protein_summary)

                analyzer.close_realtime_writers()

                print(f"✓ Analysis exported to {output_dir}")
                print(f"  Files created with timestamp: {timestamp}")

            except Exception as e:
                print(f"✗ Export failed: {e}")

        elif choice == '7':
            folder_path = input("Enter folder path containing .pdb/.pdb.gz files: ").strip()
            if folder_path and os.path.exists(folder_path):
                output_path = input("Enter output folder (press Enter for default): ").strip()
                output_path = output_path if output_path else None

                resume_choice = input("Resume from previous run if possible? (Y/n): ").strip().lower()
                resume = resume_choice != 'n'

                if resume:
                    print("\n🔄 Starting enhanced batch analysis with RESUME capability...")
                else:
                    print("\n🆕 Starting fresh batch analysis...")

                batch_summary = analyzer.batch_analyze_folder_realtime_with_resume(
                    folder_path, output_path, resume=resume
                )

                if batch_summary:
                    print(f"\n📈 Enhanced Batch Analysis Summary:")
                    print(f"  ✅ Total analyzed: {batch_summary.get('analyzed', 0)} proteins")
                    print(f"  ✗ Total failed: {batch_summary.get('failed', 0)} proteins")
                    if 'new_analyzed' in batch_summary:
                        print(f"  🆕 Newly analyzed: {batch_summary['new_analyzed']} proteins")
                        print(f"  ⏭️  Skipped (completed): {batch_summary.get('skipped_completed', 0)} proteins")
                    print(f"  📁 Results saved to: {batch_summary['output_folder']}")

                    if 'realtime_files' in batch_summary:
                        print(f"\n📊 Real-time files generated:")
                        for file_type, filename in batch_summary['realtime_files'].items():
                            print(f"  📄 {file_type.title()}: {filename}")
            else:
                print("✗ Invalid folder path")
        elif choice == '8':
            break
        else:
            print("Invalid choice. Please enter 1-8.")
if __name__ == "__main__":
    main()



⚡ Enhanced Molecular Charge Affinity Analyzer v2.1
Features:
✓ Real-time data capture during analysis
✓ Protein sequence extraction and analysis
✓ Enhanced reporting with sequence information
✓ Progress tracking and error handling
✓ RESUME capability - picks up where it left off!

Options:
1. Analyze single file (enhanced)
2. Batch analyze folder (real-time with resume)
3. Exit

Select mode (1-3): 2
Enter folder path containing .pdb/.pdb.gz files: /content/drive/MyDrive/IontheFold/downloads/ElectronMicroscope/batch_01
Enter output folder (press Enter for default): /content/drive/MyDrive/IontheFold/EDA/ElectronMicroscope/1-1000
Resume from previous run if possible? (Y/n): n

🆕 Starting fresh batch analysis...
🔍 Found 460 PDB files to analyze
📊 Analysis Summary:
  📁 Total files found: 460
  ✅ Already completed: 0
  ❌ Previously failed: 0
  🔄 Remaining to process: 460
📁 Output will be saved to: /content/drive/MyDrive/IontheFold/EDA/ElectronMicroscope/1-1000
✓ Setup real-time writer: /cont